In [153]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl import drawing 
import openpyxl

In [154]:
df = pd.read_excel("../input/horaires.xlsx", "input")
wb = load_workbook(filename="../templates/schedule_template.xlsx")

In [155]:
df["sem"] = df.semaine.apply(lambda x: "a" if "A" in x else "b")

In [156]:
template = wb["template"]

In [129]:
# get all tags in sheet
tags = {}
for row in template.iter_rows():
    for cell in row:
        if cell.value:
            if "{" in cell.value:
                tags[cell.value] = {
                    "value" : "", # free all tags content
                    "col" : cell.column,
                    "row": cell.row
                }

In [130]:
classes = df.classe.unique()

In [131]:
classe = df[df.classe == "2E"].copy()

In [132]:
# copy template
sheet = wb.copy_worksheet(wb["template"])
sheet.title = "2E"

In [133]:
# add logo
img = drawing.image.Image('../templates/Logo Ecole .jpg')
scale = 0.1
img.width = img.width * scale
img.height = img.height * scale
sheet.add_image(img, 'A1')

In [134]:
tags["{classe}"]["value"] = "2E"

In [135]:
# Populate the tags
for i, r in classe.iterrows():
    t = f"{{j{r['#day']}{r.plage.lower()}{r['sem']}"
   
    tags[t+"c}"]["value"] = r['cours'].upper()
    tags[t+"p}"]["value"] = r["pro"]
    #tags[t+"o}"]["value"] = r["code horaire"]
    tags[t+"o}"]["value"] = ""
    tags[t+"l}"]["value"] = r["/"]

In [136]:
# populate the tags in the new sheet
for t in tags:
    sheet.cell(row=tags[t]['row'], column=tags[t]['col']).value = tags[t]['value']

In [137]:
# Merge cells if needed
for i, r in classe.iterrows():
    t = f"{{j{r['#day']}{r.plage.lower()}{r['sem']}"
    
    # if AB course, get the corresponding b tag for merging
    if r.semaine == 'AB':
        tb = f"{{j{r['#day']}{r.plage.lower()}b"
        for j in ['c', 'p', 'o', 'l']:
            sheet.merge_cells(
                start_row=tags[t+j+"}"]['row'],
                start_column=tags[t+j+"}"]['col'],
                end_row=tags[tb+j+"}"]['row'],
                end_column=tags[tb+j+"}"]['col']
            )  

In [158]:
# Save result
wb.save("../output/schedule_classe.xlsx")

In [157]:
for cl in classes :
    print(cl)
    # get all tags from template
    tags = {}
    for row in template.iter_rows():
        for cell in row:
            if cell.value:
                if "{" in cell.value:
                    tags[cell.value] = {
                        "value" : "", # free all tags content
                        "col" : cell.column,
                        "row": cell.row
                    }

    
    # get the data
    classe = df[df.classe == cl].copy()
    
    classe.replace("P00", "P0", inplace=True)
    classe.replace("P01", "P1", inplace=True)
    
    # copy template
    sheet = wb.copy_worksheet(wb["template"])
    sheet.title = cl

    # add logo
    img = drawing.image.Image('../templates/Logo Ecole .jpg')
    scale = 0.1
    img.width = img.width * scale
    img.height = img.height * scale
    sheet.add_image(img, 'A1')

    # set title
    tags["{classe}"]["value"] = cl

    # Populate the tags
    for i, r in classe.iterrows():
        t = f"{{j{r['#day']}{r.plage.lower()}{r['sem']}"
       
        tags[t+"c}"]["value"] = r['cours'].upper()
        tags[t+"p}"]["value"] = r["pro"]
        #tags[t+"o}"]["value"] = r["code horaire"]
        tags[t+"o}"]["value"] = ""
        tags[t+"l}"]["value"] = r["/"]

    # populate the tags in the new sheet
    for t in tags:
        sheet.cell(row=tags[t]['row'], column=tags[t]['col']).value = tags[t]['value']

    # Merge cells if needed
    for i, r in classe.iterrows():
        t = f"{{j{r['#day']}{r.plage.lower()}{r['sem']}"
        
        # if AB course, get the corresponding b tag for merging
        if r.semaine == 'AB':
            tb = f"{{j{r['#day']}{r.plage.lower()}b"
            for j in ['c', 'p', 'o', 'l']:
                sheet.merge_cells(
                    start_row=tags[t+j+"}"]['row'],
                    start_column=tags[t+j+"}"]['col'],
                    end_row=tags[tb+j+"}"]['row'],
                    end_column=tags[tb+j+"}"]['col']
                )  

1A
1B
1C
1D
1E
2A
2B
2C
2E
2F
2G
2SA
2SB
3A
3B
3C
3D
4A
4B
4C
4D
5A
5B
6A
6B
6C
